In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.fitting import fit, common
from analysis.lib.pq import pq_tools;reload(pq_tools)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

thetas = ['pi/6']

In [ ]:
### get all timestamps!

# unshifted_days = ['20160808','20160807','20160722','20160721','20160720','20160808',\
#                   '20160807','20160722','20160724']#,'20160725']# <-- something wrong with tstamp 144023
# shifted_days = ['20160811','20160822','20160824']#,'20160823']


# all_lt3 , all_lt4 = [],[]
# offsets,offsets_ch1 = [],[] # Hold offset to compensate for timing change for new APD
# correction_time = 2772.5e3-2791.5e3
# start_offset_ch1 = 1.8e3

# for d in unshifted_days+shifted_days:
#         tstamp_lt3,tstamp_lt4 = Density_Matrix.tstamps_for_both_setups(d,contains = 'Purify')
#         #,newest_tstamp = '110000') ### newest timestamp allows for only taking parts of a day.
#         all_lt3.extend(tstamp_lt3)
#         all_lt4.extend(tstamp_lt4)
#         if d in shifted_days:
#             print 'shifting ',d
#             offsets.extend(np.zeros(np.shape(tstamp_lt3))+ correction_time)
#             offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4))+ start_offset_ch1)           
#         else:
#             offsets.extend(np.zeros(np.shape(tstamp_lt3)))
#             offsets_ch1.extend(np.zeros(np.shape(tstamp_lt4)))

In [ ]:
vals = []

for theta in thetas:
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Density_Matrix = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Density_Matrix.get_tstamps_and_offsets(contains = 'Purify')


#     ### load the data
    Density_Matrix.load_raw_data()
    Density_Matrix.correct_pq_times()

In [ ]:
### filter parameters
# photon_channel = 2 # 0 or 1; 2 means both HH detectors
# st_start = 2772.5e3#2763e3
# st_len       = 40e3 #50 ns
# st_len_w2    = 40e3
# ch1_offset = -start_offset_ch1
max_reps_w2 = 50

In [ ]:
### apply filters to data
Density_Matrix.apply_temporal_filters_to_prefiltered_data()

Density_Matrix.apply_sync_filter_w1_w2(verbose = True,max_w2 = max_reps_w2)
Density_Matrix.apply_is_purified_filter(signature = '11', verbose = True) 
Density_Matrix.apply_CR_before_filter(verbose=False)
Density_Matrix.apply_CR_after_filter(verbose=False)
Density_Matrix.attach_state_filtered_syncs(verbose = False)

In [ ]:
### acquire density matrix
Density_Matrix.get_DM_correlations(verbose = True,apply_ROC = True)
Density_Matrix.reconstruct_DMs(verbose = True,max_likelihood = False)
dm_p,dm_p_u,dm_m,dm_m_u=Density_Matrix.reconstruct_DMs(verbose = False,max_likelihood = False)

In [ ]:
reload(pa)
print dm_p
pa.plot_3D_bars(dm_p,dm_u_re = dm_p_u.real,dm_u_im = dm_p_u.imag,name='same_detector')
pa.plot_3D_bars(dm_m,dm_u_re = dm_m_u.real,dm_u_im = dm_m_u.imag,name='different_detector')

In [ ]:
import numpy as np

s1 = np.matrix([[0,1],[1,0]])
np.kron(s1,s1)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_title('Real part')
ax.bar3d?

In [ ]:
a=np.arange(0,4,1)

np.meshgrid(a,a)